# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**

# **-------------------------------------------------------------**



**TRAINING SECTION -- YOU ONLY NEED TO RUN IF YOU DONT HAVE MODEL FOLDER**

CHANGE ALL PATHS FIRST: CHANGE DATA READING PATH AND CHANGE DATA SAVING PATH

In [3]:
#!pip install pandas transformers torch datasets numpy
#!pip install accelerate -U

In [2]:
import pandas as pd
import transformers, torch

"""
Remove uneecessay new lines
remove duplicates within one cell
Remove links?
use keywords to make sure that correct classification is done
make validation set more class balanced, i.e. equal num of examples for each category
"""
# load data and rename TextEntry column
df = pd.read_excel("../../../carlos_data/preprocessed_data_v1.xlsx")
df = df.rename(columns={"TextEntry":"Description"})

# convert data to dictionary
data = df.to_dict("records")

# Split the data into train and validation and test sets
from sklearn.model_selection import train_test_split 
train_dict, test_dict = train_test_split(data, test_size=0.20, random_state=42)
test_dict, validation_dict = train_test_split(test_dict, test_size=0.50, random_state=42)

# Create Dataset objects
from datasets import Dataset
train_dataset = Dataset.from_list(train_dict)
test_dataset = Dataset.from_list(test_dict)
validation_dataset = Dataset.from_list(validation_dict)

# Create DatasetDict
from datasets import DatasetDict
dataset = DatasetDict({
	"train": train_dataset,
	"test": test_dataset,
	"validation": validation_dataset
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['ObjectID', 'Description', 'Subjective', 'Gender', 'Jargon', 'Social'],
        num_rows: 642
    })
    test: Dataset({
        features: ['ObjectID', 'Description', 'Subjective', 'Gender', 'Jargon', 'Social'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['ObjectID', 'Description', 'Subjective', 'Gender', 'Jargon', 'Social'],
        num_rows: 81
    })
})


In [4]:
# credit: https://github.com/NielsRogge/Transformers-Tutorials
labels = [label for label in dataset["train"].features.keys() if label not in ["ObjectID", "Description"]]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Subjective', 'Gender', 'Jargon', 'Social']

In [27]:
from transformers import BertTokenizer
import numpy as np

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# credit: https://github.com/NielsRogge/Transformers-Tutorials
def preprocess_data(data):

	# save the given batch of descs
	descs = data["Description"]

	# encode them using bert tokenizer
	encoding = tokenizer(descs, padding=True, truncation=True, max_length=512)#.to("mps")

	# create numpy array (no need to convert T/F to 0/1 since we annotated that way)
	# MATRIX FORMAT:
	# |---------------------------------
	# | bias   | bias1 bias2 bias3 bias4
	# |--------+------------------------
	# | desc0  |   1     0     1     0
	# | desc1  |   0     1     0     1
	# | desc2  |   0     1     0     0
	# | ...    |  ...   ...   ...   ...
	# 
	# Convert integers to float and data to an NDarray
	subjective = np.array(data["Subjective"], dtype=float)
	gender = np.array(data["Gender"], dtype=float)
	jargon = np.array(data["Jargon"], dtype=float)
	social = np.array(data["Social"], dtype=float)
	# Stack the arrays column-wise to form a 2D array (matrix)
	labels_matrix = np.stack((subjective, gender, jargon, social), axis=1)

	
	# # Credit ChatGPT
	# # Validate the data stacking by comparing 3 random indices
	# import random
	# for _ in range(3):
	# 	idx = random.randint(0, len(subjective) - 1)
	# 	dataset_labels = [data["Subjective"][idx], data["Gender"][idx], data["Jargon"][idx], data["Social"][idx]]
	# 	matrix_labels = labels_matrix[idx].tolist()
	# 	assert dataset_labels == matrix_labels, f"Mismatch at index {idx}: {dataset_labels} != {matrix_labels}"
	# 	print(f"Index {idx} matches: {dataset_labels}")


	# FORMAT OF var encoding of type BatchEncoding (the length of the vals of each key 
	# equal the num of descs/objects in given batch):
	# input_ids: [101, 1030, 4748, 7229, 1035, ...], ...
	# token_type_ids: [0, 0, 0, 0, 0, ...], ...
	# attention_mask: [1, 1, 1, 1, 1, ...], ...
	# labels: [1.0, 1.0, 0.0, 0.0], ...
	encoding["labels"] = labels_matrix.tolist()

	return encoding

In [28]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)



  0%|          | 0/1000 [06:58<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'

In [ ]:
# see example
print(tokenizer.decode(encoded_dataset["train"][0]["input_ids"]))
print(encoded_dataset["train"][0]["labels"])

[CLS] banner stone ( spearthrower weight ) location verified, inventory 2001 location verified, inventory 2003. turner inventory, april 2004. location verified, inventory 9, winter 2005. location verified, inventory 2001 location verified, inventory 2003. turner inventory, april 2004. location verified, inventory 9, winter 2005. note : updated in june 2021 with information from 1984 accession worksheet. a february 14, 1997 file memo from registrar lori iliff discussed how sources and provenance were determined for objects that were believed to have an etowah or etowah - related provenance. as x. 0250. 002 is not believed to have originated from etowah, it was not discussed in this memo. however, to document the museum records for this piece, the accession log i ( green - blue colored ledger ) does not note a provenance for x. 0250. 002 and notes the source as ferguson. x. 0250. 001 and. 002 share one specimen record that does not note a provenance and notes ferguson as source. the 1984

In [ ]:
# make dataset a standard torch dataset by converting to tensors (and more?)
encoded_dataset.set_format("torch")

In [29]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)#.to("mps")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
"""
Add Warmup?
Experiment with learning rate
experiment with batch size
experient with gradient_accumulation_steps
another metric?
Choose another optimizer: RMSprop, SGD...
Increase the learning rate by default and then use the callback ReduceLROnPlateau
"""

from transformers import TrainingArguments, Trainer

num_epochs = 50
batch_size = 32
lr = 2e-5
metric_name = "f1"
decay = 0.01

model_path = "./model"
tokenizer_path = f"./tokenizer"
logs_path = f"./logs"

with open('../../../hg_token.txt', 'r') as file:
	hg_token = file.read()


# args for training the model
# save the model every epoch and choose the best performing epoch as the final version of the model
args = TrainingArguments(
	eval_strategy = "epoch",
	save_strategy = "epoch",
	learning_rate = lr,
	per_device_train_batch_size = batch_size,
	per_device_eval_batch_size = batch_size,
	num_train_epochs = num_epochs,
	weight_decay = decay,
	load_best_model_at_end = True,
	metric_for_best_model = metric_name,
	logging_dir = logs_path,
	output_dir = model_path,
	# use_mps_device = True,
	# use_cpu = False,
	logging_steps = 1,
	gradient_accumulation_steps=2,
	hub_token = hg_token,
	hub_model_id = "raasikhk/carlos_bert_v1",
	
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
	# first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
	sigmoid = torch.nn.Sigmoid()
	probs = sigmoid(torch.Tensor(predictions))
	# next, use threshold to turn them into integer predictions
	y_pred = np.zeros(probs.shape)
	y_pred[np.where(probs >= threshold)] = 1
	# finally, compute metrics
	y_true = labels
	f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
	roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
	accuracy = accuracy_score(y_true, y_pred)
	# return as dictionary
	metrics = {'f1': f1_micro_average,
				'roc_auc': roc_auc,
				'accuracy': accuracy}
	return metrics

def compute_metrics(p: EvalPrediction):
	preds = p.predictions[0] if isinstance(p.predictions, 
			tuple) else p.predictions
	result = multi_label_metrics(
		predictions=preds, 
		labels=p.label_ids)
	return result

In [37]:
trainer = Trainer(
	model,
	args,
	train_dataset=encoded_dataset["train"],
	eval_dataset=encoded_dataset["validation"],
	tokenizer=tokenizer,
	compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

TrainOutput(global_step=500, training_loss=0.15426897712796928, metrics={'train_runtime': 3813.2838, 'train_samples_per_second': 8.418, 'train_steps_per_second': 0.131, 'total_flos': 8048711713136640.0, 'train_loss': 0.15426897712796928, 'epoch': 47.61904761904762})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.32018914818763733,
 'eval_f1': 0.6122448979591836,
 'eval_roc_auc': 0.776268115942029,
 'eval_accuracy': 0.654320987654321,
 'eval_runtime': 2.4511,
 'eval_samples_per_second': 33.047,
 'eval_steps_per_second': 1.224,
 'epoch': 47.61904761904762}

In [25]:
# view logs (only needed for analysis)
# !pip install tensorboard
!tensorboard --logdir logs

In [15]:
model.save_pretrained("save_pretrained_model")
tokenizer.save_pretrained("save_pretrained_tokenizer")
# tokenizer = transformers.BertTokenizer.from_pretrained()
# model = transformers.BertForSequenceClassification.from_pretrained()

In [10]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

logits = outputs.logits
logits.shape

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

NameError: name 'trainer' is not defined

**PREDICTION**

Run the first code block only if you have the model folder and have NOT done training above

In [18]:
model.eval()

# Whatever you want it to predict
validation_texts = ["Harsh, uncomfortably brilliant, usually reflected light.  W. April 1993 descriptor moved.", 
                    "December 1992 lead-in term added. January 1991 alternate term added. Object fumigated in Orkin's Piedmont vault with Vikane in 1994"]

# Tokenize validation texts
validation_inputs = tokenizer(validation_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

with torch.no_grad():
    outputs = model(**validation_inputs)
    logits = outputs.logits
    predictions = torch.sigmoid(logits)

# Print predictions
print(predictions)


tensor([[0.0853, 0.0375, 0.3510, 0.0492],
        [0.1108, 0.0471, 0.4768, 0.0637]])


-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------

In [24]:
# IF YOU DONT WANT TO TRAIN LOAD MODEL FROM HUGGINGFACE


from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch

device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained("raasikhk/carlos_bert_v1")
model = AutoModelForSequenceClassification.from_pretrained("raasikhk/carlos_bert_v1")

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None, device=device)

texts = [
    "The artifact was created in the 19th century and is considered highly valuable.",
    "This piece shows signs of heavy wear and might not be authentic."
]

# Make predictions
predictions = pipe(texts)

# Print the predictions
for i, text in enumerate(texts):
    print(f"Text: {text}")
    print(f"Predictions: {predictions[i]}\n")
